Trabajaremos con una base de datos sobre clientes morosos de un banco. Dentro de ésta se registran las siguientes observaciones:

- `default`​: Variable Binaria. Registra si el cliente entró en morosidad o no.
- `income​`: Ingreso promedio declarado por el cliente.
- `balance​`: total del saldo en la cuenta de crédito.
- `student​`: Variable binaria. Registra si el cliente es estudiante o no.

### Ejercicio 1: Preparación de ambiente de trabajo

- Importe los módulos básicos para el análisis de datos.
- Importe las clases `​LabelEncoder​`, `​StandardScaler` y `​LabelBinarizer` de `preprocessing`.
- Importe las funciones `​train_test_split​` y `​cross_val_score​` de `​model_selection`
- Importe la función `​classification_report​` de `​metrics​`.
- Importe las clases `​LinearDiscriminantAnalysis` y `QuadraticDiscriminantAnalysis​`.
- Agregue la base de datos en el ambiente de trabajo.
- Inspeccione la distribución de cada atributo.


In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import lec4_graphs as gfx
import classmodelsdiag as cmd
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import LabelBinarizer
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
plt.style.use( 'ggplot' )

In [2]:
df = pd.read_csv('default_credit.csv').drop(columns = 'index')
df

,default,student,balance,income
0,No,No,729.526495,44361.625074
1,No,Yes,817.180407,12106.134700
2,No,No,1073.549164,31767.138947
3,No,No,529.250605,35704.493935
4,No,No,785.655883,38463.495879
...,...,...,...,...
9995,No,No,711.555020,52992.378914
9996,No,No,757.962918,19660.721768
9997,No,No,845.411989,58636.156984
9998,No,No,1569.009053,36669.112365


In [3]:
df.describe()

,balance,income
count,10000.000000,10000.000000
mean,835.374886,33516.981876
std,483.714985,13336.639563
min,0.000000,771.967729
25%,481.731105,21340.462903
50%,823.636973,34552.644802
75%,1166.308386,43807.729272
max,2654.322576,73554.233495


In [4]:
target_label = df[ 'default' ].unique()
print(df.head())

  default student      balance        income
0      No      No   729.526495  44361.625074
1      No     Yes   817.180407  12106.134700
2      No      No  1073.549164  31767.138947
3      No      No   529.250605  35704.493935
4      No      No   785.655883  38463.495879


- Recuerde que los modelos de `​sklearn` no soportan datos que no sean numéricos. Transforme los atributos pertinentes con `​LabelEncoder​`.
- Genere muestras de validación y entrenamiento, reservando un 33% de los datos como
validación.
- Genere un modelo con `​LinearDiscriminantAnalysis` sin modificar los hiper parámetros. Genere métricas de evaluación utilizando `​classification_report​`.
- Comente sobre cuál es el desempeño del modelo en cada clase, así como en general.

In [5]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
df[ 'default' ] = LabelEncoder().fit_transform(df[ 'default' ])
df[ 'student' ] = LabelEncoder().fit_transform(df[ 'student' ])

In [6]:
df.describe()

,default,student,balance,income
count,10000.000000,10000.000000,10000.000000,10000.000000
mean,0.033300,0.294400,835.374886,33516.981876
std,0.179428,0.455795,483.714985,13336.639563
min,0.000000,0.000000,0.000000,771.967729
25%,0.000000,0.000000,481.731105,21340.462903
50%,0.000000,0.000000,823.636973,34552.644802
75%,0.000000,1.000000,1166.308386,43807.729272
max,1.000000,1.000000,2654.322576,73554.233495


In [7]:
X = df[['student', 'balance', 'income']]
y = df['default']
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.33, random_state=8874)


In [8]:
lda = LinearDiscriminantAnalysis()
lda.fit(X_train, y_train)


LinearDiscriminantAnalysis()

In [9]:
y_pred = lda.predict(X_val)


In [10]:
from sklearn.metrics import classification_report
report = classification_report(y_val, y_pred)
print(report)


              precision    recall  f1-score   support

           0       0.97      1.00      0.98      3185
           1       0.71      0.19      0.30       115

    accuracy                           0.97      3300
   macro avg       0.84      0.59      0.64      3300
weighted avg       0.96      0.97      0.96      3300



El desempeño en general es bueno, en la clase 0 es muy bueno pero el desempeño en general se ve afectado por el de la clase 1.

- Dado que trabajamos con modelos generativos, podemos incluir información exógena. Para este caso agregaremos dos distribuciones:
    + Asumamos que hay un 50/50 de morosos y no morosos.
    + Asumamos que hay un 60/40 de morosos y no morosos.
- Por cada modelo, reporte las métricas de clasificación.

In [11]:
from sklearn.datasets import make_classification
from sklearn.metrics import classification_report
X_50, y_50 = make_classification(n_samples=1000, n_features=4, weights=[0.5, 0.5], random_state=8874)
X_60, y_60 = make_classification(n_samples=1000, n_features=4, weights=[0.6, 0.4], random_state=8874)


In [12]:
X_train_50, X_val_50, y_train_50, y_val_50 = train_test_split(X_50, y_50, test_size=0.33, random_state=8874)
X_train_60, X_val_60, y_train_60, y_val_60 = train_test_split(X_60, y_60, test_size=0.33, random_state=8874)


In [13]:
lda_50 = LinearDiscriminantAnalysis()
lda_60 = LinearDiscriminantAnalysis()
lda_50.fit(X_train_50, y_train_50)
lda_60.fit(X_train_60, y_train_60)


LinearDiscriminantAnalysis()

In [14]:
y_pred_50 = lda_50.predict(X_val_50)
y_pred_60 = lda_60.predict(X_val_60)


In [15]:
report_50 = classification_report(y_val_50, y_pred_50)
report_60 = classification_report(y_val_60, y_pred_60)

print("Reporte de clasificación - Distribución 50/50:")
print(report_50)
print("\nReporte de clasificación - Distribución 60/40:")
print(report_60)


Reporte de clasificación - Distribución 50/50:
              precision    recall  f1-score   support

           0       0.90      0.87      0.88       170
           1       0.87      0.89      0.88       160

    accuracy                           0.88       330
   macro avg       0.88      0.88      0.88       330
weighted avg       0.88      0.88      0.88       330


Reporte de clasificación - Distribución 60/40:
              precision    recall  f1-score   support

           0       0.92      0.91      0.91       201
           1       0.86      0.88      0.87       129

    accuracy                           0.90       330
   macro avg       0.89      0.89      0.89       330
weighted avg       0.90      0.90      0.90       330



Refactorización 2 - oversampling

#### Digresión: Synthetic Over(Under)Sampling

- Por lo general podemos intentar aliviar el problema del desbalance de clases mediante la ponderación dentro del algoritmo. Otra alternativa es el muestreo con reemplazo dentro de los conjuntos de entrenamiento. Estos métodos clásicos se conocen como **Oversampling** cuando repetimos registros aleatorios de la clase minoritaria, y **Undersampling​** cuando eliminamos aleatoriamente registros de la clase mayoritaria.

- Un contratiempo de estos métodos clásicos es que pueden replicar información sesgada que afecte el desempeño de generalización del modelo. Si los datos son malos, estaremos replicando estas fallas.

- Otra solución es generar ejemplos de entrenamiento sintéticos mediante el entrenamiento de ejemplos de la clase minoritaria. A grandes rasgos la solución funciona de la siguiente forma: En función a un subconjunto de datos correspondientes a la clase minoritaria, entrenamos algún modelo no supervisado o generativo como Naive Bayes, KMeans o KNearestNeighbors para generar representaciones sintéticas de
los datos **​en el espacio de atributos de la clase específica** mediante $x_\text{nuevo−ejemplo} = x_i + \lambda(x_{zi} − x_i)$ es un ejemplo de entrenamiento de la clase minoritaria y $\lambda$ es un parámetro de interpolación aleatorio $\lambda ~ \text{Uniforme}(0, 1)$.

- Uno de los problemas más graves de esta base de datos, s el fuerte desbalance entre clases. Ahora generaremos observaciones sintéticas mediante SMOTE (Synthetic Minority Oversampling Technique). Para ello, debemos agregar el paquete a nuestro ambiente virtual. En nuestro terminal agregamos `​conda install -c conda-forge imbalanced-learn`​. Incorpore SMOTE en el ambiente de trabajo con la siguiente sintáxis `​from​ imblearn.over_sampling ​import​ SMOTE`​.

- Para implementar oversampling, debemos generar nuevos objetos que representan nuestra muestra de entrenamiento incrementada artificialmente. Para ello implemente la siguiente sintaxis:

```python	
from​ imblearn.over_sampling ​import​ SMOTE
# Instanciamos la clase
oversampler = SMOTE(random_state=​11238​, ratio=​'minority'​)
# generamos el eversampling de la matriz de entrenamiento y
X_train_oversamp, y_train_oversamp = oversampler.fit_resample(X_train, y_train)
```	
- Vuelva a entrenar el modelo con los datos aumentados de forma artificial y comente sobre su desempeño.

In [16]:
from imblearn.over_sampling import SMOTE
oversampler = SMOTE(random_state=11238, sampling_strategy='minority')
X_train_oversamp, y_train_oversamp = oversampler.fit_resample(X_train, y_train)


In [17]:
lda_oversamp = LinearDiscriminantAnalysis()
lda_oversamp.fit(X_train_oversamp, y_train_oversamp)
y_pred_oversamp = lda_oversamp.predict(X_val)


In [18]:
report_oversamp = classification_report(y_val, y_pred_oversamp)
print(report_oversamp)


              precision    recall  f1-score   support

           0       0.99      0.85      0.92      3185
           1       0.17      0.85      0.28       115

    accuracy                           0.85      3300
   macro avg       0.58      0.85      0.60      3300
weighted avg       0.97      0.85      0.89      3300



en similitud al caso anterior, el desempeño de la clase 0 sigue bien, pero el desempeño de la clase 1 esta muy deficiente.

Refactorización 3 - QDA

- Por último, implemente un modelo `​QuadraticDiscriminantAnalysis` con los datos aumentados artificialmente. Genere las métricas de desempeño.

- Comente a grandes rasgos sobre el mejor modelo en su capacidad predictiva.

In [19]:
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
qda = QuadraticDiscriminantAnalysis()
#entrenamiento
qda.fit(X_train_oversamp, y_train_oversamp)
y_pred_qda = qda.predict(X_val)
report_qda = classification_report(y_val, y_pred_qda)
print(report_qda)


              precision    recall  f1-score   support

           0       0.99      0.88      0.93      3185
           1       0.19      0.80      0.31       115

    accuracy                           0.88      3300
   macro avg       0.59      0.84      0.62      3300
weighted avg       0.96      0.88      0.91      3300

